In [ ]:
pip install scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/d5/2c/734fc9269bdb6768905ac41b82d75264b26925b1e462f4ebf45fe4f17646/scikit_learn-1.5.1-cp312-cp312-macosx_12_0_arm64.whl.metadata
  Obtaining dependency information for scipy>=1.6.0 from https://files.pythonhosted.org/packages/5c/76/f2b91ea2d2b76504e845699271be9c0ca3492770614fb6b911fb517023de/scipy-1.14.0-cp312-cp312-macosx_12_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 203.7 kB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for joblib>=1.2.0 from https://files.pythonhosted.org/packages/91/29/df4b9b42f2be0b623cbd5e2140cafcaa2bef0759a00b7b70104dcfe2fb51/joblib-1.4.2-py3-none-any.whl.metadata
  Obtaining dependency information for threadpoolctl>=3.1.0 from https://files.pythonhosted.org/packages/4b/2c/ffbf7a134b9ab11a67b0cf0726453cedd9c5043a4fe7a35d1cefa9a1bcfb/threadpoolctl-3.5.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Read coordinates from CSV
csv_file = '/Users/joshmanto/Downloads/BB/tree-semantics/locations.csv'
df = pd.read_csv(csv_file)
df.columns = df.columns.str.strip()  # Strip spaces from column names
df = df[['Longitude (dd)', 'Latitude (dd)']]
df.columns = ['longitude', 'latitude']  # Rename columns for consistency
df.head()

# Perform clustering to identify regions
coords = df[['longitude', 'latitude']].values
db = DBSCAN(eps=0.01, min_samples=10).fit(coords)
df['region'] = db.labels_

# Print the number of clusters found
num_clusters = len(set(df['region'])) - (1 if -1 in df['region'] else 0)
print(f'Number of clusters found: {num_clusters}')
df.head()


KeyError: "None of [Index(['Longitude (dd)', 'Latitude (dd)'], dtype='object')] are in the [columns]"

In [ ]:
from shapely.geometry import Polygon

# Define AOI using the coordinates from the CSV file
def define_aoi(coordinates):
    coords = list(zip(coordinates['longitude'], coordinates['latitude']))
    AOI = {
        "type": "Polygon",
        "coordinates": [coords]
    }
    return AOI

# Calculate the bounding box for the AOI and expand it to create the AOA
def calculate_aoa(coordinates, expansion_factor=0.01):
    min_lon, max_lon = coordinates['longitude'].min(), coordinates['longitude'].max()
    min_lat, max_lat = coordinates['latitude'].min(), coordinates['latitude'].max()
    
    # Expand the bounding box slightly
    min_lon -= expansion_factor
    max_lon += expansion_factor
    min_lat -= expansion_factor
    max_lat += expansion_factor
    
    AOA = {
        "type": "Polygon",
        "coordinates": [[
            [min_lon, min_lat],
            [min_lon, max_lat],
            [max_lon, max_lat],
            [max_lon, min_lat],
            [min_lon, min_lat]
        ]]
    }
    return AOA

# Group coordinates by region and calculate AOI and AOA for each region
regions = df['region'].unique()
region_aois_aoas = {}
for region in regions:
    if region == -1:
        continue  # Ignore noise points
    region_coords = df[df['region'] == region]
    AOI = define_aoi(region_coords)
    AOA = calculate_aoa(region_coords)
    region_aois_aoas[region] = {'AOI': AOI, 'AOA': AOA}

region_aois_aoas


In [ ]:
from shapely.geometry import shape

# Determine if AOI is inside AOA
def is_inside(AOI, AOA):
    smaller_shape = shape(AOI)
    larger_shape = shape(AOA)
    return smaller_shape.within(larger_shape)

# Check if AOI is inside AOA for each region
for region, aoi_aoa in region_aois_aoas.items():
    AOI = aoi_aoa['AOI']
    AOA = aoi_aoa['AOA']
    result = is_inside(AOI, AOA)
    print(f"Region {region} - AOI is within AOA: {result}")


In [ ]:
import os
import requests

# Set up the API key and endpoint
os.environ['PL_API_KEY'] = 'PLAKe8a128a493104644888a58e5a0b4b780'
PLANET_API_KEY = os.getenv('PL_API_KEY')
BASE_URL = "https://api.planet.com/analytics/"

# Set up a session and authenticate
session = requests.Session()
session.auth = (PLANET_API_KEY, "")

# Test the connection
res = session.get(BASE_URL)
print(res.status_code)  # Should print 200 if the connection is successful
print(res.text)  # Print the response body

# Create a subscription
def create_subscription(name, AOI):
    payload = {
        "name": name,
        "aoi": AOI,
        "filter": {
            "type": "AndFilter",
            "config": [
                {"type": "GeometryFilter", "field_name": "geometry", "config": AOI},
                {"type": "DateRangeFilter", "field_name": "acquired", "config": {"gte": "2022-01-01T00:00:00.000Z", "lte": "2022-12-31T23:59:59.999Z"}}
            ]
        }
    }
    response = session.post(f"{BASE_URL}/subscriptions", json=payload)
    response.raise_for_status()
    return response.json()

# Create subscriptions for each region if AOI is within AOA
for region, aoi_aoa in region_aois_aoas.items():
    AOI = aoi_aoa['AOI']
    AOA = aoi_aoa['AOA']
    if is_inside(AOI, AOA):
        subscription = create_subscription(f"Subscription for region {region}", AOI)
        print(f"Created Subscription for region {region}:", subscription)
    else:
        print(f"AOI for region {region} is not within AOA")
